In [5]:
import pandas as pd
import os
# !pip install xlrd

In [6]:
# loading data sets
countries_names_to_code = pd.read_csv(os.path.join('data',r'countryarea.csv'))[['Country Name','Country Code']]

votes = pd.read_csv(os.path.join('data',r'votes_with_minorities.csv'))
votes.drop(columns=['Unnamed: 0'],inplace=True)
print(countries_names_to_code.head())
print(len(countries_names_to_code))
print(votes.head())
print(len(votes))

  Country Name Country Code
0        Aruba          ABW
1  Afghanistan          AFG
2       Angola          AGO
3      Albania          ALB
4      Andorra          AND
265
   Year (semi-) final Jury or Televoting From country To country  Points  \
0  1975             f                  J      Belgium    Belgium       0   
1  1976             f                  J      Belgium    Belgium       0   
2  1977             f                  J      Belgium    Belgium       0   
3  1978             f                  J      Belgium    Belgium       0   
4  1979             f                  J      Belgium    Belgium       0   

   2_minority_in_1  
0              0.0  
1              0.0  
2              0.0  
3              0.0  
4              0.0  
38181


In [7]:
# adding code to FROM countries
votes_with_code = pd.merge(votes, countries_names_to_code, how='left', left_on=['From country'], right_on=['Country Name'])
votes_with_code.drop(columns=['Country Name'],inplace=True)
votes_with_code.rename(columns={"Country Code": "From country code"},inplace=True)
print(len(votes_with_code.index))

# adding code to TO countries
votes_with_code = pd.merge(votes_with_code, countries_names_to_code, how='left', left_on=['To country'], right_on=['Country Name'])
votes_with_code.drop(columns=['Country Name'],inplace=True)
votes_with_code.rename(columns={"Country Code": "To country code"},inplace=True)
votes_with_code = votes_with_code[['Year', '(semi-) final', 'Jury or Televoting','From country', 'From country code', 'To country', 
                   'To country code','Points', '2_minority_in_1']]
len(votes_with_code.index)

38181


38181

In [8]:
# loading distances 
countries_dist = pd.read_excel(os.path.join('data',r'countries_distw.xls'))[['iso_o','iso_d','distw']]
countries_dist.rename(columns={"iso_o": "From country code", "iso_d": "To country code"},inplace=True)
countries_dist.head()

,From country code,To country code,distw
0,ABW,ABW,25.0935
1,ABW,AFG,13168.2
2,ABW,AGO,9587.32
3,ABW,AIA,976.897
4,ABW,ALB,9091.58


In [15]:
votes_with_distance = pd.merge(votes_with_code, countries_dist, how='left', on=['From country code',"To country code"])
votes_with_distance.drop(columns=['From country code', "To country code"],inplace=True)
null_countries = ['Yugoslavia','Serbia & Montenegro','Monaco','Romania','Montenegro','Serbia']
votes_with_distance = votes_with_distance[(~votes_with_distance['From country'].isin(null_countries))]
votes_with_distance = votes_with_distance[(~votes_with_distance['To country'].isin(null_countries))]

votes_with_distance.to_csv(os.path.join('data',r'votes_with_dist.csv'))

print(votes_with_distance.groupby(['From country']).apply(lambda grp: grp.isnull().sum()))
print('-------------------------------------------')
nulls = votes_with_distance.isnull().sum()
print(nulls, nulls/len(votes_with_distance))
# 4098 null == 0.1%

# don't have dist and ISO: Serbia & Montenegro, Yugoslavia 
# don't have dist: Monaco, Romania

                      Year  (semi-) final  Jury or Televoting  From country  \
From country                                                                  
Albania                  0              0                   0             0   
Andorra                  0              0                   0             0   
Armenia                  0              0                   0             0   
Australia                0              0                   0             0   
Austria                  0              0                   0             0   
Azerbaijan               0              0                   0             0   
Belarus                  0              0                   0             0   
Belgium                  0              0                   0             0   
Bosnia                   0              0                   0             0   
Bosnia & Herzegovina     0              0                   0             0   
Bulgaria                 0              0           